In [2]:
import arviz as az
import jax.numpy as jnp
import matplotlib.pyplot as plt
import numpy as np
import numpyro
import numpyro.distributions as dist
from jax import random
from numpyro.infer import MCMC, NUTS

# Kotaro used the following to generation the mock data
N = 20
for i in range(5):
    for j in range(2):
        _shape, _rate = random.random(), random.random()

        concentration = _shape * jnp.ones(N)
        rate = _rate * jnp.ones(N)
        s = dist.Gamma(concentration=concentration, rate=rate).sample(random.PRNGKey(0))



# list(np.array(s))

[2.3762975,
 2.7325578,
 1.3750422,
 0.9992071,
 1.9519926,
 6.251218,
 0.84728867,
 6.076127,
 6.0899625,
 2.6973338,
 3.7665267,
 1.572763,
 6.4568233,
 1.369886,
 6.5856237,
 1.5776805,
 7.356719,
 7.4597907,
 3.3213968,
 4.0831566]

In [3]:
# Data that I generated using an arbitrary pair of (shape, rate) parameters.
d = [2.3762975,
     2.7325578,
     1.3750422,
     0.9992071,
     1.9519926,
     6.251218,
     0.84728867,
     6.076127,
     6.0899625,
     2.6973338,
     3.7665267,
     1.572763,
     6.4568233,
     1.369886,
     6.5856237,
     1.5776805,
     7.356719,
     7.4597907,
     3.3213968,
     4.0831566]
data = jnp.array(d)


In [4]:
# Write the probabilistic model to estimate the shape and rate parameters.
def gamma_model(data):
    alpha = numpyro.sample("alpha", dist.Exponential(1.0))
    beta = numpyro.sample("beta", dist.Exponential(1.0))
    g = numpyro.sample("g", dist.Gamma(alpha, beta), obs=data)

mcmc = MCMC(NUTS(model=gamma_model), num_warmup=1000, num_samples=1000)
mcmc.run(random.PRNGKey(0), data=s)
mcmc.print_summary()
#                mean       std    median      5.0%     95.0%     n_eff     r_hat
#     alpha      2.18      0.55      2.14      1.23      3.00    283.12      1.00
#      beta      0.59      0.16      0.58      0.30      0.83    279.37      1.00

sample: 100%|██████████| 2000/2000 [00:01<00:00, 1234.43it/s, 3 steps of size 3.19e-01. acc. prob=0.94] 



                mean       std    median      5.0%     95.0%     n_eff     r_hat
     alpha      2.18      0.55      2.14      1.23      3.00    283.12      1.00
      beta      0.59      0.16      0.58      0.30      0.83    279.37      1.00

Number of divergences: 0


In [ ]:
import csv
file = open('workload.csv')
csvreader = csv.reader(file)

header = []
header = next(csvreader)
rows = []
for row in csvreader:
  rows.append(row)
file.close()

data_ = []
for i in range(2):
  data_list = []
  for j in rows:
    if int(j[2]) == i:
      data_list.append(float(j[1]))
  data_.append(data_list)

for i in range(2):


  d = data_[i]
  data = jnp.array(d)

  # Write the probabilistic model to estimate the shape and rate parameters.
  def gamma_model(data):
      alpha = numpyro.sample("alpha", dist.Exponential(1.0))
      beta = numpyro.sample("beta", dist.Exponential(1.0))
      g = numpyro.sample("g", dist.Gamma(alpha, beta), obs=data)

  mcmc = MCMC(NUTS(model=gamma_model), num_warmup=1000, num_samples=10000)
  mcmc.run(random.PRNGKey(0), data=data)
  mcmc.print_summary()
  

In [ ]:
import pyomo.environ as pyo

m = pyo.ConcreteModel('example')
m.I = pyo.Set(initialize=list(range(len(data))))
m.A = RangeSet(0, len(data) -1)
m.x = pyo.Var(m.I,m.I, domain=pyo.Binary)
m.delay = pyo.Var(m.I)
m.y=pyo.Var(m.I,m.I, domain=pyo.Binary)

deadlines = [5, 20, 15, 10, 15]
workloads = [3, 5, 5, 6, 6]

def rule_one(mdl, i):
    return mdl.x[i, i] == 1

m.budget = Constraint(m.A, rule=rule_one)

def rule_two(mdl, i):
    return mdl.delay[i] >= 0 

m.two = Constraint(m.A, rule=rule_two)

def rule_three(mdl, i):
    return mdl.delay[i] >= sum(mdl.x[j,i]*workloads[j] for j in range(len(data))) - deadlines[i]

m.three = Constraint(m.A, rule=rule_three)

def rule_four(mdl, i, k):
  if i == k:
    return mdl.x[i,i] >= 0
  else:
    return sum(mdl.x[j,i]*workloads[j] for j in range(len(data))) - sum(mdl.x[j,k]*workloads[j] for j in range(len(data))) - workloads[k] <= (1 - mdl.x[i,k]) * 1000

m.four = Constraint(m.A,m.A, rule=rule_four)

def rule_five(mdl, i, k):
  if i == k:
    return mdl.x[i,i] >= 0
  else:
    return sum(mdl.x[j,i]*workloads[j] for j in range(len(data))) - sum(mdl.x[j,k]*workloads[j] for j in range(len(data))) >= - mdl.x[i,k] * 1000

m.five = Constraint(m.A,m.A, rule=rule_five)

def rule_six(mdl, i, k):
  if i == k:
    return mdl.x[i,i] >= 0
  else:
    return mdl.x[i,k] + mdl.x[k,i] == 1

m.six = Constraint(m.A,m.A, rule=rule_six)

# Objective function
def objective_func(mdl):
    return sum(mdl.delay[i] for i in range(len(data)))
m.objective = pyo.Objective(rule=objective_func, sense=pyo.minimize)

SolverFactory('cbc').solve(m).write()
